<hr><center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Stop untagged EC2 Instances</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Stop-Untagged-EC2-Instances&para;"><u>Stop Untagged EC2 Instances</u></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview</h1>
<p>1)&nbsp;<a href="#1" target="_self" rel="noopener">Get all untagged EC2 instance</a><br>2)&nbsp;<a href="#2" target="_self" rel="noopener">Stop Untagged EC2 instances</a></p>
<hr>

In [ ]:
if region == None:
    region = ''
if instance_ids and not region:
    raise SystemExit("Provide a region for the EC2 Instance IDs!")

<h3 id="Get-publicly-accessible-DB-snapshots"><a id="2" target="_self" rel="nofollow"></a>Get all Untagged EC2 Instances</h3>
<p>Here we will fetch all the untagged&nbsp; EC2 instances.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>untagged_instances</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Tuple, Optional
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_filter_untagged_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def check_untagged_instance(res, r):
    instance_list = []
    for reservation in res:
        for instance in reservation['Instances']:
            instances_dict = {}
            tags = instance.get('Tags', None)
            if tags is None:
                instances_dict['region']= r
                instances_dict['instanceID']= instance['InstanceId']
                instance_list.append(instances_dict)
    return instance_list


@beartype
def aws_filter_untagged_ec2_instances(handle, region: str= None) -> Tuple:
    """aws_filter_untagged_ec2_instances Returns an array of instances which has no tags.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter instances.

        :rtype: Tuple of status, and list of untagged EC2 Instances
    """
    result = []
    all_instances = []
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=r)
            res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
            untagged_instances = check_untagged_instance(res, r)
            if len(untagged_instances)!=0:
                all_instances.append(untagged_instances)
        except Exception as e:
            pass
    try:
        result = all_instances[0]
    except Exception as e:
        pass
    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not instance_ids",
    "condition_result": true
    }''')
task.configure(outputName="untagged_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_untagged_ec2_instances, lego_printer=aws_filter_untagged_ec2_instances_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-untagged-instances"><a id="2" target="_self" rel="nofollow"></a>Create List of untagged instances<a class="jp-InternalAnchorLink" href="#Create-List-of-untagged-instances" target="_self">&para;</a></h3>
<blockquote>
<p>This action captures the following output: <code>all_untagged_instances</code></p>
</blockquote>

In [5]:
all_untagged_instances = []
try:
    if untagged_instances[0] == False:
        if len(untagged_instances[1])!=0:
            all_untagged_instances=untagged_instances[1]
except Exception:
    for ids in instance_ids:
        data_dict = {}
        data_dict["region"] = region
        data_dict["instanceID"] = ids
        all_untagged_instances.append(data_dict)
print(all_untagged_instances)
task.configure(outputName="all_untagged_instances")

<h3 id="Get-publicly-accessible-DB-snapshots"><a id="2" target="_self" rel="nofollow"></a>Stop untagged EC2 Instances</h3>
<p>Using unSkript's Stop EC2 instances action we will stop all untagged EC2 instances found in Step 1.

<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_stop_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_stop_instances(handle, instance_id: str, region: str) -> Dict:
    """aws_stop_instances Stops instances.

        :type instance_id: string
        :param instance_id: String containing the name of AWS EC2 instance

        :type region: string
        :param region: AWS region for instance

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    output = {}
    res = ec2Client.stop_instances(InstanceIds=[instance_id])
    for instances in res['StoppingInstances']:
        output[instances['InstanceId']] = instances['CurrentState']

    return output


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "instance_id": "iter.get(\\"instanceID\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_untagged_instances",
    "iter_parameter": ["instance_id","region"]
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_untagged_instances)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_stop_instances, lego_printer=aws_stop_instances_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Conclusion&para;" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to find all untagged EC2 instances and stop them using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>